# Ensemble

In [ ]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=284595d83ff1d63c7e431a59c2a1fa16f084e9c4c3b22be5d2428dbcf496c2ba
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/Colab'

/content/gdrive/MyDrive/Colab


In [ ]:
import import_ipynb
import os
import sklearn.metrics as metrics
import Utils as utils
import tensorflow as tf
import numpy as np

importing Jupyter notebook from Utils.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Models

In [ ]:
#Load best models

scratch = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab/bestmodels/Copia di scratch_mc_32.h5')

inception = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab/bestmodels/Copia di inception_mc_16_3.h5')

vgg = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab/bestmodels/Copia di vgg_mc_64_2.h5')

## Data preprocessing

In [ ]:
test_images, test_labels = utils.load_test()
test_dataset, test_lab = utils.binarize(test_images, test_labels, 'mass_calcification')

In [ ]:
#Scratch
test_scratch = utils.rescale(test_dataset)
test_scratch = np.expand_dims(test_scratch, axis=3)

#Inception 

test_inception = utils.rescale(test_dataset)
test_inception = [np.stack((test_inception[i],)*3, axis=2) for i in range (len(test_inception))]

test_inception = np.array(test_inception)

## Evaluation

In [ ]:
#Scratch

predicted_s = scratch.predict(test_scratch, verbose=1)
pred_s = [0 if x <=0.5 else 1 for x in predicted_s]

#Inception 
predicted_i = inception.predict(test_inception, verbose=1)
pred_i = [0 if x <=0.5 else 1 for x in predicted_i]

#Vgg
predicted_v = vgg.predict(test_inception, verbose=1)
pred_v = [0 if x <=0.5 else 1 for x in predicted_v]

11/11 [==============================] - 1s 73ms/step


In [ ]:
#Compute AUC Score

score_scratch = metrics.roc_auc_score(test_lab, pred_s, average="weighted")
score_inception = metrics.roc_auc_score(test_lab, pred_i, average="weighted")
score_vgg = metrics.roc_auc_score(test_lab, pred_v, average="weighted")

print("Scratch Score: ", score_scratch)
print("Inception Score: ", score_inception)
print("VGG Score: ", score_vgg)


Scratch Score:  0.8716685051417998
Inception Score:  0.9142974059708928
VGG Score:  0.8764188876632388


In [ ]:
predicted_s = np.reshape(predicted_s, -1)
predicted_i = np.reshape(predicted_i, -1)
predicted_v = np.reshape(predicted_v, -1)

ensemble_results = 0.33* (predicted_s*score_scratch + predicted_i*score_inception + predicted_v*score_vgg)
ensemble_results = [0 if x <=0.5 else 1 for x in ensemble_results]

utils.evaluate_classifier(test_lab, ensemble_results)


Performances: 
Accuracy:  0.9017857142857143
Precision:  0.9017615486365487
Recall:  0.9017857142857143
F1-score:  0.9017656111542661
AUC:  0.9011671351812973
